## Library

In [ ]:
import os
import time
import logging
import warnings
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from pydantic import BaseModel, Field
from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
)

from sentence_transformers import SentenceTransformer
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.callbacks import get_openai_callback
from langchain_huggingface import HuggingFaceEmbeddings

from dotenv import load_dotenv
load_dotenv()

warnings.filterwarnings("ignore")

In [ ]:
seed = 1
retrieve_k = 3

## Data

In [ ]:
train_df = pd.read_csv("train_형사.csv")
val_df = pd.read_csv("validation_형사.csv")
test_df = pd.read_csv("test_형사.csv")

combined_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

datasets = {}

test_size = 1000
train_data, test_data = train_test_split(
    combined_df,
    test_size=test_size,
    stratify=combined_df["answer"],
    random_state=seed,
)

val_size = 100
train_data, validation_data = train_test_split(
    train_data,
    test_size=val_size,
    stratify=train_data["answer"],
    random_state=seed,
)

datasets[seed] = {
    "train_data": train_data,
    "test_data": test_data,
    "validation_data": validation_data,
}

train = datasets[seed]["train_data"]
validation = datasets[seed]["validation_data"]
test = datasets[seed]["test_data"]

## Vectorstore

In [ ]:
device = torch.device("cpu")

class KUREEmbedding:
    def __init__(self, model_name="nlpai-lab/KURE-v1"):
        self.model = SentenceTransformer(model_name, trust_remote_code=True).to(device)

    def embed_documents(self, texts):
        embeddings = self.model.encode(texts, convert_to_numpy=True)
        return embeddings

    def embed_query(self, text):
        return self.embed_documents([text])[0]

class KoE5Embedding(KUREEmbedding):
    def __init__(self, model_name="nlpai-lab/KoE5"):
        super().__init__(model_name)

In [ ]:
vectorstore_path = f"embedding_comparison/faiss_index_koe5_seed{seed}"
if os.path.exists(vectorstore_path):
    embeddings = KoE5Embedding()

    vectorstore = FAISS.load_local(
        vectorstore_path,
        embeddings.embed_query,
        allow_dangerous_deserialization=True,
    )

    retriever = vectorstore.as_retriever(search_kwargs={"k": retrieve_k})

## Classifier

In [ ]:
class StatementClassification(BaseModel):
    predicted_label: str = Field(description="Predicted law classification label for the statement True, False")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
structured_llm_labeler = llm.with_structured_output(StatementClassification)

## Prompt & Chain

In [ ]:
multiquery_prompt = ChatPromptTemplate.from_messages([
        ("system", "Generate three alternative versions of the given query in Korean, each within 50 characters."),
        ("human", "Original query: {question}")
    ])
 
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm=llm,
    prompt=multiquery_prompt
)

classification_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Classify the query into one of : True , False  \n
                Return in JSON: {{\"prediction\": \"category\"}}
            """
         ),
        (
            "human",
            "reference document: \n\n {similar_cases} \n\n Input document: {statement}",
        ),
    ]
)

## Multi-Query

In [ ]:
def predict_label_with_multiquery_rag(statement: str, idx: int, total: int, max_docs: int = 3):

    queries = [statement]
    
    formatted_prompt = multiquery_prompt.format(question=statement)
    generated_queries = llm.invoke(formatted_prompt).content.strip().split("\n")
    queries.extend(generated_queries)
    
    all_retrieved_docs = []
    
    for query in queries:
        retrieved_docs = retriever_from_llm.invoke(query)[:max_docs]
        all_retrieved_docs.extend(retrieved_docs)
        
    unique_docs = list({doc.page_content: doc for doc in all_retrieved_docs}.values())

    retrieved_texts_str = "\n".join([doc.page_content for doc in unique_docs])
    
    formatted_prompt = classification_prompt.format(statement=statement, similar_cases=retrieved_texts_str)
    
    prediction = structured_llm_labeler.invoke(formatted_prompt)
    
    return prediction

## Prediction

In [ ]:
results = []

for idx, row in tqdm(test.iterrows(), total=len(test), desc="Processing"):
    question = row["question"]
    actual_label = row["answer"]
    
    prediction = predict_label_with_multiquery_rag(question, idx+1, len(test))

    results.append(
        {           
            "question": question,
            "answer": actual_label,
            "prediction": prediction.predicted_label,
        }
    )

df_results = pd.DataFrame(results)

## Evaluation

In [ ]:
y_true = df_results["answer"].map({True: 1, False: 0})
y_pred = df_results["prediction"].map({"True": 1, "False": 0})

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1_final = f1_score(y_true, y_pred)
f1_macro = f1_score(y_true, y_pred, average="macro")
f1_weighted = f1_score(y_true, y_pred, average="weighted")

conf_matrix = confusion_matrix(y_true, y_pred)

print("\n===== Classification Performance Results =====")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_final:.4f}")
print(f"F1-score (Macro): {f1_macro:.4f}")
print(f"F1-score (Weighted): {f1_weighted:.4f}")

print("\n===== Classification Confusion Matrix =====")
print(conf_matrix)

print("\n===== Detailed Classification Report =====")
print(classification_report(y_true, y_pred))